In [1]:
import pandas as pd
from utils import utils

data = pd.read_csv("reconciled_articles.csv", sep = "\t")
data_tissues = utils.explode_commas(data, "Tissue")

                                DOI      Organism                Tissue  \
0           10.1073/PNAS.97.11.6144           Rat                 Brain   
1     10.1016/S0092-8674(00)80581-4         Mouse                 Brain   
2     10.1016/S0896-6273(03)00229-0  Human, Mouse                 Brain   
3             10.1093/CERCOR/BHJ081           Rat                 Brain   
4                10.1093/NAR/GKL050         Mouse                   ICM   
...                             ...           ...                   ...   
1025     10.1016/J.STEM.2020.11.015         Human  [Bone marrow, Liver]   
1026   10.1016/J.IMMUNI.2020.12.003         Human                  Lung   
1027   10.1371/JOURNAL.PBIO.3001017         Mouse               Culture   
1029     10.1016/J.CELL.2020.12.016         Human             Intestine   
1030   10.1016/J.CELREP.2020.108573         Human           Bone marrow   

               Technique Data location  \
0             sc-RT-mPCR           NaN   
1              

/home/lubianat/Documents/wiki_related/wikidata_singlecell/utils/utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = strings


In [3]:
tissues = pd.Series(list(set(data_tissues['Tissue'])))
tissues.to_csv("tissues_to_reconcile.csv")

 I did the reconciliation via Google Sheets and the Wikidata add on. Worked nicely.

In [8]:
reconciled_tissues = pd.read_csv("reconciled_tissues.csv")
tissue2wikidata = {}
for i, row in reconciled_tissues.iterrows():
    tissue2wikidata[row["0"]] = row["match"]

In [10]:
data_tissues["tissue_qid"] = data_tissues["Tissue"].map(tissue2wikidata)

In [11]:
with open("project_uses_tissue_08_02_2021.qs", "w") as f:
    for i, row in data_tissues.iterrows():
        s = row["item"]
        p = "P4510" # external data available at
        o = row["tissue_qid"]
        rp1 = "S248"
        ro1 = "Q103034964"
        rp2 = "S854"
        ro2 = '"https://www.nxn.se/single-cell-studies/data.tsv"'
        rp4 = "S813"
        ro4 = "+2021-02-07T00:00:00Z/11"
        f.write(f"{s}|{p}|{o}|{rp1}|{ro1}|{rp2}|{ro2}|{rp4}|{ro4}")
        f.write("\n")